# 🎵 Spotify Analytics Pipeline - Démonstration Complète

Ce notebook documente et démontre les deux pipelines ETL du projet d'analyse de données Spotify.

## 📋 Table des Matières

1. [Introduction et Configuration](#introduction)
2. [Pipeline 1 : CSV → Oracle → XML → HTML](#pipeline1)
3. [Pipeline 2 : XML → XSD → JSON → MongoDB](#pipeline2)
4. [Analyse et Visualisation des Données](#analyse)
5. [Requêtes Avancées MongoDB](#requetes)
6. [Conclusion](#conclusion)

---

## 1. Introduction et Configuration <a id="introduction"></a>

### Architecture Globale

Ce projet implémente **deux pipelines ETL complémentaires** :

**Pipeline 1 : Relationnel (Oracle)**
```
CSV → Normalisation → Oracle → XML → Validation DTD → XSLT → HTML Dashboard
```

**Pipeline 2 : NoSQL (MongoDB)**
```
XML → Génération XSD → Validation XSD → XSLT → JSON → MongoDB
```

### Imports et Configuration

In [ ]:
# Imports standards
import sys
import os
import json
from pathlib import Path
import pandas as pd
from datetime import datetime

# Ajouter le répertoire du projet au PYTHONPATH
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"📁 Répertoire de travail : {project_root}")
print(f"🐍 Version Python : {sys.version}")
print(f"📅 Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Imports des modules du projet
try:
    from DB.db_manager import DatabaseManager
    from DB.mongodb_manager import MongoDBManager
    from services.data_processor import preprocess_csv
    from services.xml_exporter import export_to_xml
    from services.dtd_creator import create_spotify_dtd
    from services.dtd_validator import validate_xml_with_dtd
    from services.xsd_creator import create_spotify_xsd
    from services.xsd_validator import validate_xml_with_xsd
    from services.json_converter import convert_xml_to_json
    from services.xslt_transformer import transform_to_html
    from configs.config import *
    
    print("✅ Tous les modules ont été importés avec succès !")
except ImportError as e:
    print(f"❌ Erreur d'import : {e}")
    print("💡 Assurez-vous d'exécuter ce notebook depuis le répertoire racine du projet")

### Vérification des Fichiers et Dossiers

In [ ]:
# Vérifier la structure du projet
print("📂 Structure du projet :\n")

directories = [
    "configs",
    "DB",
    "services",
    "data/input",
    "data/output"
]

for directory in directories:
    path = Path(directory)
    status = "✅" if path.exists() else "❌"
    print(f"{status} {directory}")

print("\n📄 Fichiers importants :\n")

files = [
    "data/input/high_popularity_spotify_data.csv",
    "data/input/spotify_transform.xslt",
    "data/input/spotify_to_json.xslt",
    "main.py",
    "requirements.txt"
]

for file in files:
    path = Path(file)
    status = "✅" if path.exists() else "❌"
    print(f"{status} {file}")

---

## 2. Pipeline 1 : CSV → Oracle → XML → HTML <a id="pipeline1"></a>

### Étape 1.1 : Extraction et Normalisation des Données CSV

In [ ]:
print("🔄 Extraction des données CSV...\n")

# Prétraiter le CSV
data_to_insert = preprocess_csv()

if data_to_insert:
    print("\n✅ Données extraites et normalisées !\n")
    print("📊 Aperçu des tables créées :\n")
    
    for table_name, df in data_to_insert.items():
        print(f"  • {table_name:<25} : {len(df):>6} lignes")
else:
    print("❌ Échec de l'extraction")

### Visualisation des Données Extraites

In [ ]:
# Afficher un échantillon de chaque table
if data_to_insert:
    print("📋 Échantillons de données :\n")
    
    # Genres
    print("\n=== GENRES ===")
    display(data_to_insert['sp_genres'].head(5))
    
    # Playlists
    print("\n=== PLAYLISTS ===")
    display(data_to_insert['sp_playlists'].head(5))
    
    # Tracks
    print("\n=== TRACKS ===")
    display(data_to_insert['sp_tracks'].head(5))
    
    # Audio Features
    print("\n=== AUDIO FEATURES ===")
    display(data_to_insert['sp_audio_features'].head(5))

### Étape 1.2 : Test de Connexion à Oracle

In [ ]:
print("🔌 Test de connexion à Oracle...\n")

db_manager = DatabaseManager()

if db_manager.connect():
    print("\n✅ Connexion Oracle établie !")
    
    # Afficher les statistiques si des tables existent
    try:
        stats = db_manager.get_statistics()
        if stats:
            print("\n📊 Tables existantes :")
            for table, count in stats.items():
                print(f"  • {table:<25} : {count:>6} lignes")
    except:
        print("\nℹ️  Aucune table existante (base vide)")
    
    db_manager.close()
else:
    print("❌ Échec de la connexion Oracle")
    print("💡 Vérifiez que Oracle est démarré et les identifiants sont corrects")

### Étape 1.3 : Insertion des Données dans Oracle

⚠️ **Attention** : Cette cellule supprime et recrée les tables existantes !

In [ ]:
# DÉCOMMENTER POUR EXÉCUTER (supprime les données existantes !)
# print("🔄 Initialisation de la base de données Oracle...\n")

# db_manager = DatabaseManager()

# if db_manager.connect():
#     # Initialiser (drop + create tables)
#     print("⚠️  Suppression et recréation des tables...")
#     db_manager.initialize_db(drop_first=True)
    
#     # Insérer les données
#     print("\n💾 Insertion des données...")
#     success = db_manager.insert_data(data_to_insert)
    
#     if success:
#         print("\n✅ Données insérées avec succès !")
        
#         # Afficher les statistiques
#         stats = db_manager.get_statistics()
#         if stats:
#             print("\n📊 Statistiques finales :")
#             for table, count in stats.items():
#                 print(f"  • {table:<25} : {count:>6} lignes")
#     else:
#         print("❌ Échec de l'insertion")
    
#     db_manager.close()

print("ℹ️  Cellule commentée pour éviter la suppression accidentelle des données")
print("💡 Décommentez le code pour exécuter l'insertion")

### Étape 1.4 : Export vers XML

In [ ]:
print("🔄 Export des données vers XML...\n")

db_manager = DatabaseManager()

if db_manager.connect():
    # Récupérer les données pour l'export
    xml_data = db_manager.fetch_data_for_xml()
    
    if xml_data:
        print(f"✅ {len(xml_data)} enregistrements récupérés\n")
        
        # Exporter vers XML
        xml_file = export_to_xml(xml_data)
        
        if xml_file:
            print(f"\n✅ Fichier XML généré : {xml_file}")
            
            # Afficher la taille du fichier
            file_size = Path(xml_file).stat().st_size
            print(f"📦 Taille : {file_size:,} bytes ({file_size/1024:.2f} KB)")
    else:
        print("❌ Aucune donnée à exporter")
    
    db_manager.close()
else:
    print("❌ Échec de la connexion Oracle")

### Étape 1.5 : Validation DTD

In [ ]:
print("🔄 Création et validation DTD...\n")

# Créer la DTD
dtd_file = create_spotify_dtd()

if dtd_file:
    print("\n✅ DTD créée avec succès\n")
    
    # Valider le XML
    xml_file = XML_OUTPUT_PATH
    
    if Path(xml_file).exists():
        is_valid, errors = validate_xml_with_dtd(xml_file, dtd_file)
        
        if is_valid:
            print("\n✅ Validation DTD réussie !")
        else:
            print(f"\n❌ Validation échouée : {len(errors)} erreur(s)")
    else:
        print(f"❌ Fichier XML introuvable : {xml_file}")
else:
    print("❌ Échec de la création de la DTD")

### Étape 1.6 : Transformation XSLT → HTML

In [ ]:
print("🔄 Transformation XSLT vers HTML...\n")

xml_file = XML_OUTPUT_PATH

if Path(xml_file).exists():
    html_file = transform_to_html(xml_file)
    
    if html_file:
        print(f"\n✅ Dashboard HTML généré : {html_file}")
        
        # Afficher la taille
        file_size = Path(html_file).stat().st_size
        print(f"📦 Taille : {file_size:,} bytes ({file_size/1024:.2f} KB)")
        
        print("\n💡 Ouvrez le fichier dans votre navigateur pour voir le dashboard")
    else:
        print("❌ Échec de la transformation HTML")
else:
    print(f"❌ Fichier XML introuvable : {xml_file}")

---

## 3. Pipeline 2 : XML → XSD → JSON → MongoDB <a id="pipeline2"></a>

### Étape 2.1 : Génération du Schéma XSD

In [ ]:
print("🔄 Génération du schéma XSD...\n")

xsd_file = XSD_PATH

success = create_spotify_xsd(xsd_file)

if success:
    print("\n✅ Schéma XSD créé avec succès !")
    
    # Afficher la taille
    file_size = Path(xsd_file).stat().st_size
    print(f"📦 Taille : {file_size:,} bytes ({file_size/1024:.2f} KB)")
else:
    print("❌ Échec de la création du XSD")

### Étape 2.2 : Validation XSD

In [ ]:
print("🔄 Validation du XML avec le schéma XSD...\n")

xml_file = XML_OUTPUT_PATH
xsd_file = XSD_PATH

if Path(xml_file).exists() and Path(xsd_file).exists():
    is_valid, errors = validate_xml_with_xsd(xml_file, xsd_file)
    
    if is_valid:
        print("\n✅ Validation XSD réussie !")
        print("📋 Le XML est conforme au schéma XSD")
    else:
        print(f"\n❌ Validation échouée : {len(errors)} erreur(s)")
        for error in errors[:5]:  # Afficher les 5 premières erreurs
            print(f"  • Ligne {error.get('line', 'N/A')} : {error.get('message', 'N/A')}")
else:
    if not Path(xml_file).exists():
        print(f"❌ Fichier XML introuvable : {xml_file}")
    if not Path(xsd_file).exists():
        print(f"❌ Fichier XSD introuvable : {xsd_file}")

### Étape 2.3 : Transformation XML → JSON via XSLT

In [ ]:
print("🔄 Transformation XML → JSON via XSLT...\n")

xml_file = XML_OUTPUT_PATH
xslt_file = XSLT_JSON_PATH
json_file = JSON_OUTPUT_PATH

if Path(xml_file).exists() and Path(xslt_file).exists():
    success, json_data = convert_xml_to_json(xml_file, xslt_file, json_file)
    
    if success and json_data:
        print("\n✅ Conversion JSON réussie !")
        
        # Afficher des statistiques
        print(f"\n📊 Statistiques du JSON :")
        print(f"  • Total playlists    : {json_data.get('total_playlists', 0)}")
        print(f"  • Total tracks       : {json_data.get('total_tracks', 0)}")
        
        # Taille du fichier
        file_size = Path(json_file).stat().st_size
        print(f"  • Taille fichier     : {file_size:,} bytes ({file_size/1024:.2f} KB)")
    else:
        print("❌ Échec de la conversion JSON")
else:
    if not Path(xml_file).exists():
        print(f"❌ Fichier XML introuvable : {xml_file}")
    if not Path(xslt_file).exists():
        print(f"❌ Fichier XSLT introuvable : {xslt_file}")

### Aperçu du JSON Généré

In [ ]:
# Charger et afficher un échantillon du JSON
json_file = JSON_OUTPUT_PATH

if Path(json_file).exists():
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print("📋 Aperçu du JSON (première playlist) :\n")
    
    if data.get('playlists'):
        first_playlist = data['playlists'][0]
        
        # Afficher avec indentation
        print(json.dumps(first_playlist, indent=2, ensure_ascii=False)[:1000] + "...")
else:
    print(f"❌ Fichier JSON introuvable : {json_file}")

### Étape 2.4 : Test de Connexion MongoDB

In [ ]:
print("🔌 Test de connexion à MongoDB...\n")

mongo_manager = MongoDBManager(
    host=MONGO_HOST,
    port=MONGO_PORT,
    database=MONGO_DATABASE
)

if mongo_manager.connect():
    print("\n✅ Connexion MongoDB établie !")
    
    # Lister les collections existantes
    try:
        collections = mongo_manager.db.list_collection_names()
        if collections:
            print("\n📊 Collections existantes :")
            for collection in collections:
                count = mongo_manager.db[collection].count_documents({})
                print(f"  • {collection:<20} : {count:>6} documents")
        else:
            print("\nℹ️  Aucune collection existante (base vide)")
    except Exception as e:
        print(f"⚠️  Erreur : {e}")
    
    mongo_manager.close()
else:
    print("❌ Échec de la connexion MongoDB")
    print(f"💡 Vérifiez que MongoDB est démarré sur {MONGO_HOST}:{MONGO_PORT}")

### Étape 2.5 : Insertion des Données dans MongoDB

⚠️ **Attention** : Cette cellule supprime et recrée la collection `playlists` !

In [ ]:
print("🔄 Insertion des données dans MongoDB...\n")

json_file = JSON_OUTPUT_PATH

if not Path(json_file).exists():
    print(f"❌ Fichier JSON introuvable : {json_file}")
else:
    # Charger le JSON
    with open(json_file, 'r', encoding='utf-8') as f:
        json_data = json.load(f)
    
    # Se connecter à MongoDB
    mongo_manager = MongoDBManager(
        host=MONGO_HOST,
        port=MONGO_PORT,
        database=MONGO_DATABASE
    )
    
    if mongo_manager.connect():
        # Insérer les playlists (avec suppression de la collection existante)
        success, count = mongo_manager.insert_spotify_playlists(json_data, clear_first=True)
        
        if success:
            print(f"\n✅ {count} playlists insérées avec succès !")
            
            # Afficher les statistiques
            stats = mongo_manager.get_collection_stats('playlists')
        else:
            print("❌ Échec de l'insertion")
        
        mongo_manager.close()
    else:
        print("❌ Échec de la connexion MongoDB")

---

## 4. Analyse et Visualisation des Données <a id="analyse"></a>

### Analyse des Données dans MongoDB

In [ ]:
# Imports pour la visualisation
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    # Configuration du style
    plt.style.use('seaborn-v0_8-darkgrid')
    sns.set_palette("husl")
    
    print("✅ Bibliothèques de visualisation chargées")
except ImportError as e:
    print(f"⚠️  Erreur d'import : {e}")
    print("💡 Installez les dépendances : pip install matplotlib seaborn")

In [ ]:
print("📊 Récupération des données depuis MongoDB...\n")

mongo_manager = MongoDBManager(
    host=MONGO_HOST,
    port=MONGO_PORT,
    database=MONGO_DATABASE
)

if mongo_manager.connect():
    # Récupérer toutes les playlists
    collection = mongo_manager.db['playlists']
    playlists = list(collection.find({}))
    
    if playlists:
        print(f"✅ {len(playlists)} playlists récupérées\n")
        
        # Convertir en DataFrame pour l'analyse
        playlists_df = pd.DataFrame(playlists)
        
        print("📋 Colonnes disponibles :")
        print(playlists_df.columns.tolist())
        
        print("\n📊 Statistiques de base :")
        print(f"  • Total playlists    : {len(playlists_df)}")
        print(f"  • Genres uniques     : {playlists_df['genre'].nunique()}")
        print(f"  • Subgenres uniques  : {playlists_df['subgenre'].nunique()}")
        
        # Total de tracks
        total_tracks = playlists_df['tracks_count'].sum()
        print(f"  • Total tracks       : {total_tracks}")
        print(f"  • Tracks/playlist (moy) : {total_tracks/len(playlists_df):.2f}")
    else:
        print("⚠️  Aucune playlist trouvée")
    
    mongo_manager.close()
else:
    print("❌ Échec de la connexion MongoDB")

### Distribution des Genres

In [ ]:
if 'playlists_df' in locals():
    # Distribution des genres
    genre_counts = playlists_df['genre'].value_counts()
    
    plt.figure(figsize=(12, 6))
    genre_counts.plot(kind='bar', color='skyblue')
    plt.title('Distribution des Playlists par Genre', fontsize=16, fontweight='bold')
    plt.xlabel('Genre', fontsize=12)
    plt.ylabel('Nombre de Playlists', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Top 10 des genres :")
    for genre, count in genre_counts.head(10).items():
        print(f"  • {genre:<20} : {count:>3} playlists")
else:
    print("❌ Données non disponibles")

### Distribution des Tracks par Playlist

In [ ]:
if 'playlists_df' in locals():
    plt.figure(figsize=(12, 6))
    plt.hist(playlists_df['tracks_count'], bins=20, color='lightcoral', edgecolor='black')
    plt.title('Distribution du Nombre de Tracks par Playlist', fontsize=16, fontweight='bold')
    plt.xlabel('Nombre de Tracks', fontsize=12)
    plt.ylabel('Nombre de Playlists', fontsize=12)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Statistiques sur les tracks :")
    print(f"  • Minimum  : {playlists_df['tracks_count'].min()} tracks")
    print(f"  • Maximum  : {playlists_df['tracks_count'].max()} tracks")
    print(f"  • Moyenne  : {playlists_df['tracks_count'].mean():.2f} tracks")
    print(f"  • Médiane  : {playlists_df['tracks_count'].median():.0f} tracks")
else:
    print("❌ Données non disponibles")

### Analyse des Caractéristiques Audio

Extraction et analyse des caractéristiques audio moyennes par genre.

In [ ]:
if 'playlists' in locals():
    # Extraire les caractéristiques audio de tous les tracks
    audio_features_list = []
    
    for playlist in playlists:
        genre = playlist.get('genre', 'unknown')
        for track in playlist.get('tracks', []):
            features = track.get('audio_features', {})
            if features:
                audio_features_list.append({
                    'genre': genre,
                    'energy': features.get('energy', 0),
                    'tempo': features.get('tempo', 0),
                    'danceability': features.get('danceability', 0),
                    'loudness': features.get('loudness', 0),
                    'valence': features.get('valence', 0)
                })
    
    audio_df = pd.DataFrame(audio_features_list)
    
    print(f"✅ {len(audio_df)} caractéristiques audio extraites\n")
    
    # Moyennes par genre
    audio_by_genre = audio_df.groupby('genre').mean()
    
    print("📊 Caractéristiques audio moyennes par genre :\n")
    display(audio_by_genre.round(3))
else:
    print("❌ Données non disponibles")

### Heatmap des Caractéristiques Audio par Genre

In [ ]:
if 'audio_by_genre' in locals():
    # Normaliser les valeurs pour une meilleure visualisation
    audio_normalized = audio_by_genre.copy()
    audio_normalized['tempo'] = audio_normalized['tempo'] / audio_normalized['tempo'].max()
    audio_normalized['loudness'] = (audio_normalized['loudness'] - audio_normalized['loudness'].min()) / (audio_normalized['loudness'].max() - audio_normalized['loudness'].min())
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(audio_normalized.T, annot=True, fmt='.2f', cmap='YlOrRd', cbar_kws={'label': 'Valeur Normalisée'})
    plt.title('Heatmap des Caractéristiques Audio par Genre\n(Valeurs normalisées)', fontsize=16, fontweight='bold')
    plt.xlabel('Genre', fontsize=12)
    plt.ylabel('Caractéristique', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print("❌ Données non disponibles")

---

## 5. Requêtes Avancées MongoDB <a id="requetes"></a>

### Exemples de Requêtes MongoDB

#### Requête 1 : Playlists par Genre

In [ ]:
print("🔍 Recherche de playlists latines...\n")

mongo_manager = MongoDBManager(
    host=MONGO_HOST,
    port=MONGO_PORT,
    database=MONGO_DATABASE
)

if mongo_manager.connect():
    collection = mongo_manager.db['playlists']
    
    # Requête : playlists du genre "latin"
    latin_playlists = list(collection.find(
        {"genre": "latin"},
        {"nom": 1, "subgenre": 1, "tracks_count": 1, "_id": 0}
    ).limit(10))
    
    if latin_playlists:
        print(f"✅ {len(latin_playlists)} playlists trouvées :\n")
        
        latin_df = pd.DataFrame(latin_playlists)
        display(latin_df)
    else:
        print("⚠️  Aucune playlist trouvée")
    
    mongo_manager.close()
else:
    print("❌ Échec de la connexion MongoDB")

#### Requête 2 : Tracks les Plus Populaires

In [ ]:
print("🔍 Recherche des tracks les plus populaires...\n")

mongo_manager = MongoDBManager(
    host=MONGO_HOST,
    port=MONGO_PORT,
    database=MONGO_DATABASE
)

if mongo_manager.connect():
    collection = mongo_manager.db['playlists']
    
    # Agrégation : dérouler les tracks et trier par popularité
    pipeline = [
        {"$unwind": "$tracks"},
        {"$sort": {"tracks.popularity": -1}},
        {"$limit": 20},
        {"$project": {
            "_id": 0,
            "track_name": "$tracks.name",
            "artist": "$tracks.artist.name",
            "popularity": "$tracks.popularity",
            "genre": "$genre"
        }}
    ]
    
    top_tracks = list(collection.aggregate(pipeline))
    
    if top_tracks:
        print(f"✅ Top 20 des tracks les plus populaires :\n")
        
        top_df = pd.DataFrame(top_tracks)
        display(top_df)
    else:
        print("⚠️  Aucun track trouvé")
    
    mongo_manager.close()
else:
    print("❌ Échec de la connexion MongoDB")

#### Requête 3 : Playlists avec Beaucoup de Tracks

In [ ]:
print("🔍 Recherche des playlists avec plus de 20 tracks...\n")

mongo_manager = MongoDBManager(
    host=MONGO_HOST,
    port=MONGO_PORT,
    database=MONGO_DATABASE
)

if mongo_manager.connect():
    collection = mongo_manager.db['playlists']
    
    # Requête : playlists avec tracks_count > 20
    big_playlists = list(collection.find(
        {"tracks_count": {"$gt": 20}},
        {"nom": 1, "genre": 1, "tracks_count": 1, "_id": 0}
    ).sort("tracks_count", -1))
    
    if big_playlists:
        print(f"✅ {len(big_playlists)} playlists trouvées :\n")
        
        big_df = pd.DataFrame(big_playlists)
        display(big_df)
    else:
        print("⚠️  Aucune playlist trouvée")
    
    mongo_manager.close()
else:
    print("❌ Échec de la connexion MongoDB")

#### Requête 4 : Statistiques Agrégées par Genre

In [ ]:
print("🔍 Calcul des statistiques par genre...\n")

mongo_manager = MongoDBManager(
    host=MONGO_HOST,
    port=MONGO_PORT,
    database=MONGO_DATABASE
)

if mongo_manager.connect():
    collection = mongo_manager.db['playlists']
    
    # Agrégation : statistiques par genre
    pipeline = [
        {"$group": {
            "_id": "$genre",
            "nombre_playlists": {"$sum": 1},
            "total_tracks": {"$sum": "$tracks_count"},
            "tracks_moyen": {"$avg": "$tracks_count"}
        }},
        {"$sort": {"nombre_playlists": -1}}
    ]
    
    stats_by_genre = list(collection.aggregate(pipeline))
    
    if stats_by_genre:
        print(f"✅ Statistiques par genre :\n")
        
        stats_df = pd.DataFrame(stats_by_genre)
        stats_df.rename(columns={'_id': 'genre'}, inplace=True)
        stats_df['tracks_moyen'] = stats_df['tracks_moyen'].round(2)
        display(stats_df)
    else:
        print("⚠️  Aucune statistique disponible")
    
    mongo_manager.close()
else:
    print("❌ Échec de la connexion MongoDB")

---

## 6. Conclusion <a id="conclusion"></a>

### Résumé du Projet

Ce notebook a démontré l'implémentation complète de deux pipelines ETL complémentaires :

#### 🔷 Pipeline 1 : Architecture Relationnelle (Oracle)
- ✅ Extraction et normalisation de données CSV
- ✅ Stockage structuré dans Oracle Database
- ✅ Export XML avec validation DTD
- ✅ Génération de dashboard HTML interactif

#### 🔶 Pipeline 2 : Architecture NoSQL (MongoDB)
- ✅ Validation XML avec schéma XSD strict
- ✅ Transformation XSLT vers JSON
- ✅ Stockage de documents hiérarchiques dans MongoDB
- ✅ Requêtes flexibles et agrégations puissantes

### Avantages de la Double Architecture

| Aspect | Oracle (Relationnel) | MongoDB (NoSQL) |
|--------|---------------------|------------------|
| Structure | Normalisée (3NF) | Hiérarchique (documents imbriqués) |
| Intégrité | Contraintes strictes | Schéma flexible |
| Requêtes | SQL complexes avec jointures | Agrégations MongoDB natives |
| Performance | Optimisé pour transactions | Optimisé pour lecture/agrégations |
| Use Case | Rapports analytiques précis | Exploration de données flexible |

### Technologies Maîtrisées

- 🐍 **Python** : Orchestration ETL, traitement de données
- 🗄️ **Oracle** : Base de données relationnelle
- 🍃 **MongoDB** : Base de données NoSQL
- 📄 **XML/XSD/DTD** : Formats de données structurées
- 🔄 **XSLT** : Transformation de données
- 📊 **Pandas** : Manipulation de données
- 📈 **Matplotlib/Seaborn** : Visualisation de données

### Fichiers Générés

✅ **data/output/spotify_data_export.xml** - Export XML depuis Oracle  
✅ **data/output/spotify_data.dtd** - Schéma DTD  
✅ **data/output/spotify_data.xsd** - Schéma XSD  
✅ **data/output/spotify_data.json** - Export JSON  
✅ **data/output/spotify_data.html** - Dashboard interactif  
✅ **MongoDB : spotify_db.playlists** - 72 documents imbriqués

### Commandes Utiles pour Aller Plus Loin

#### MongoDB Shell
```javascript
// Connexion
use spotify_db

// Compter les documents
db.playlists.countDocuments()

// Recherche par nom
db.playlists.find({ nom: /Rock/i })

// Tracks avec haute énergie
db.playlists.aggregate([
  { $unwind: "$tracks" },
  { $match: { "tracks.audio_features.energy": { $gt: 0.8 } } },
  { $project: { 
      track: "$tracks.name",
      artist: "$tracks.artist.name",
      energy: "$tracks.audio_features.energy"
  }},
  { $sort: { energy: -1 } },
  { $limit: 10 }
])
```

#### CLI Python
```bash
# Pipeline complet Oracle
python main.py --full-reset

# Pipeline complet MongoDB
python main.py --mongodb-pipeline

# Tests de connexion
python main.py --test-connection
python main.py --test-mongodb
```

---

### 🎉 Fin du Notebook

Ce notebook a démontré l'utilisation complète des deux pipelines ETL pour l'analyse de données Spotify.

**Auteur** : Cash  
**Projet** : DBA_Spotify Analytics Pipeline  
**Date** : Décembre 2025

---